In [4]:
!git clone --recursive https://github.com/Microsoft/LightGBM
! cd LightGBM && rm -rf build && mkdir build && cd build && cmake -DUSE_GPU=1 ../../LightGBM && make -j4 && cd ../python-package && python3 setup.py install --precompile --gpu;

from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedKFold, cross_val_score
import warnings
warnings.filterwarnings(action='ignore')
from xgboost import XGBRegressor
!pip install catboost
import catboost

import pandas as pd
from tqdm.auto import tqdm
import numpy as np
from datetime import date, timedelta
import time
from google.colab import drive
drive.mount('/content/drive')
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler, PowerTransformer,QuantileTransformer ,LabelEncoder, OneHotEncoder, MinMaxScaler, OrdinalEncoder, RobustScaler
import seaborn as sns
! pip install prince
import prince

! pip install optuna
import optuna
from optuna import Trial, visualization
from optuna.samplers import TPESampler
from sklearn.metrics import mean_squared_error
import lightgbm as lgb

Cloning into 'LightGBM'...
remote: Enumerating objects: 119, done.
remote: Counting objects: 100% (119/119), done.
remote: Compressing objects: 100% (81/81), done.
remote: Total 21413 (delta 69), reused 63 (delta 37), pack-reused 21294
Receiving objects: 100% (21413/21413), 16.85 MiB | 27.96 MiB/s, done.
Resolving deltas: 100% (15678/15678), done.
Submodule 'include/boost/compute' (https://github.com/boostorg/compute) registered for path 'external_libs/compute'
Submodule 'eigen' (https://gitlab.com/libeigen/eigen.git) registered for path 'external_libs/eigen'
Submodule 'external_libs/fast_double_parser' (https://github.com/lemire/fast_double_parser.git) registered for path 'external_libs/fast_double_parser'
Submodule 'external_libs/fmt' (https://github.com/fmtlib/fmt.git) registered for path 'external_libs/fmt'
Cloning into '/content/LightGBM/external_libs/compute'...
remote: Enumerating objects: 21728, done.        
remote: Total 21728 (delta 0), reused 0 (delta 0), pack-reused 21728 

In [5]:
path = '/content/drive/Shareddrives/dacon/Playground/'
train = pd.read_csv(path + 'train.csv', index_col=['id'])
test = pd.read_csv(path + 'test.csv', index_col=['id'])
submission = pd.read_csv(path + 'sample_submission.csv', index_col=['id'])

In [6]:
cat_features = [feature for feature in train.columns if 'cat' in feature]
cont_features = [feature for feature in train.columns if 'cont' in feature]

In [7]:
train_data = train.iloc[:,:-1]
train_target = train.iloc[:, -1]
test_data = test.copy()

In [8]:
# # CAT4, CAT6 Drop하는 경우
# train_temp = train_data.drop(['cat6'], axis = 1)
# test_temp = test_data.drop(['cat6'], axis = 1)

train_temp = train_data.copy()
test_temp = test_data.copy()

cat_features = [feature for feature in train_temp.columns if 'cat' in feature]
cont_features = [feature for feature in train_temp.columns if 'cont' in feature]

# scaler  = QuantileTransformer(random_state=42, n_quantiles=2500)
scaler  = PowerTransformer()

a = pd.get_dummies(train_temp[cat_features])
b = pd.get_dummies(test_temp[cat_features])
missing_cols = set(a.columns) - set(b.columns)
for c in missing_cols:
  b[c] = 0
b = b[a.columns]

train_temp2 = train_data[cont_features]
test_temp2 = test_data[cont_features]
train_sc = scaler.fit_transform(train_temp2)
test_sc = scaler.transform(test_temp2)

train_scaled = pd.DataFrame(train_sc, index = train_temp2.index, columns = train_temp2.columns)
test_scaled = pd.DataFrame(test_sc, index = test_temp2.index, columns = test_temp2.columns)

trainset = pd.concat([train_scaled, a, train_target], axis = 1)
testset = pd.concat([test_scaled, b], axis = 1)

def droper(df):
  trainset = df.copy()
  trainset.drop(['cat3_B', 'cat8_B'], axis = 1, inplace=True)

  drops_cat5 = ['cat5_B', 'cat5_A', 'cat5_D']
  trainset['cat5_ABD'] = trainset[drops_cat5].sum(axis = 1)
  trainset.drop(drops_cat5, axis = 1, inplace=True)

  drops_cat6 = ['cat6_E', 'cat6_I', 'cat6_H']
  trainset['cat6_EIH'] = trainset[drops_cat6].sum(axis = 1)
  trainset.drop(drops_cat6, axis = 1, inplace=True)


  drops_cat7_1 = ['cat7_G', 'cat7_E']
  trainset['cat7_GE'] = trainset[drops_cat7_1].sum(axis = 1)
  drops_cat7_2 = ['cat7_B', 'cat7_D']
  trainset['cat7_BD'] = trainset[drops_cat7_2].sum(axis = 1)
  trainset.drop(drops_cat7_1, axis = 1, inplace=True)
  trainset.drop(drops_cat7_2, axis = 1, inplace=True)

  drops_cat8 = ['cat8_A', 'cat8_D']
  trainset['cat8_AD'] = trainset[drops_cat8].sum(axis = 1)
  trainset.drop(drops_cat8, axis = 1, inplace=True)


  drops_cat9_1 = ['cat9_B', 'cat9_O', 'cat9_K']
  trainset['cat9_BOK'] = trainset[drops_cat9_1].sum(axis = 1)
  drops_cat9_2 = ['cat9_N', 'cat9_M', 'cat9_H']
  trainset['cat9_NMH'] = trainset[drops_cat9_2].sum(axis = 1)
  trainset.drop(drops_cat9_1, axis = 1, inplace=True)
  trainset.drop(drops_cat9_2, axis = 1, inplace=True)
  return trainset

In [9]:
def create_stratified_folds_for_regression(data_df, n_splits=10):
    """
    @param data_df: training data to split in Stratified K Folds for a continous target value
    @param n_splits: number of splits
    @return: the training data with a column with kfold id
    """
    data_df['kfold'] = -1
    # randomize the data
    data_df = data_df.sample(frac=1, random_state=42).reset_index(drop=True)
    # calculate the optimal number of bins based on log2(data_df.shape[0])
    num_bins = np.int(np.floor(1 + np.log2(len(data_df))))
    # bins value will be the equivalent of class value of target feature used by StratifiedKFold to 
    # distribute evenly the classed over each fold
    data_df.loc[:, "bins"] = pd.cut(pd.to_numeric(data_df['target'], downcast="signed"), bins=num_bins, labels=False)
    kf = StratifiedKFold(n_splits=n_splits, random_state=42)
    
    # set the fold id as a new column in the train data
    for f, (t_, v_) in enumerate(kf.split(X=data_df, y=data_df.bins.values)):
        data_df.loc[v_, 'kfold'] = f
    
    # drop the bins column (no longer needed)
    data_df = data_df.drop("bins", axis=1)
    
    return data_df

In [10]:
trainset2 = droper(trainset)
testset2 = droper(testset)

In [11]:
trainset3 = trainset2.copy()
trainset3.drop(['target'], axis = 1, inplace = True)

In [12]:
# Outlier 제거
from sklearn.ensemble import IsolationForest
iso_model = IsolationForest(max_samples='auto', random_state=42, n_jobs = -1, contamination = 0.1, n_estimators=500)
iso_model.fit(trainset3)

score = iso_model.decision_function(trainset3)
anomaly = iso_model.predict(trainset3)
trainset3['scores'] = score
trainset3['anomaly'] = anomaly

score2 = iso_model.decision_function(testset2)
anomaly2 = iso_model.predict(testset2)
testset2['scores'] = score2
testset2['anomaly'] = anomaly2

In [13]:
trainset4 = pd.concat([trainset3, train_target], axis = 1)
train_df = create_stratified_folds_for_regression(trainset4)

In [27]:
def cv_evaluation_regression2(df, model1, model2, model3, test):
    n_fold = 10
    oof = np.zeros((len(df),))

    oof_lgb = np.zeros((len(df),))
    oof_xgb = np.zeros((len(df),))
    oof_cat = np.zeros((len(df),))

    lgbm = 0
    xgb = 0
    cat = 0
    a = 5
    b = 1
    c = 4
    for fold in tqdm(range(n_fold)):
        val_ind = df[df.kfold == fold].index
        train = df[df.kfold != fold].reset_index(drop=True)
        valid = df[df.kfold == fold].reset_index(drop=True)

        features = [feature for feature in df.columns if feature not in ['id', 'target', 'kfold']]

        X_train = train[features]
        y_train = train['target']
        X_valid = valid[features]
        y_valid = valid['target']

        print('LGBM')
        model1.fit(X_train, y_train, eval_set = [(X_valid, y_valid)], early_stopping_rounds = 500, verbose=2000)
        print('XGB')
        model2.fit(X_train, y_train, eval_set = [(X_valid, y_valid)], early_stopping_rounds = 250, verbose=500)
        print('CATBOOST')
        model3.fit(X_train, y_train, eval_set = [(X_valid, y_valid)], early_stopping_rounds = 500, verbose=2000)

        preds1 = model1.predict(valid[features])
        preds2 = model2.predict(valid[features])
        preds3 = model3.predict(valid[features])

        lgbm1 = model1.predict(test[features])
        xgb1 = model2.predict(test[features])
        cat1 = model3.predict(test[features])

        pred = (a * preds1 + b*preds2 + c*preds3)/(a + b +c)
        oof[val_ind] = pred
        oof_lgb[val_ind] = preds1
        oof_xgb[val_ind] = preds2
        oof_cat[val_ind] = preds3

        lgbm += lgbm1
        xgb += xgb1
        cat += cat1

    mse_oof = mean_squared_error(oof, df['target'])
    lgbm = lgbm/n_fold
    xgb = xgb/n_fold
    cat = cat/n_fold
    return np.sqrt(mse_oof), lgbm, xgb, cat, oof_lgb, oof_xgb, oof_cat

In [28]:
def objective(X, test_data):

  params = {} #initialize parameters
  params['random_state'] = 42
  params['learning_rate'] = 0.001
  params['n_estimators'] = 30000
  params['device_type'] = 'gpu'
  params['feature_fraction'] = 0.8073718294893066
  params['subsample'] = 0.5090670592304201
  params['num_leaves'] = 235
  params['min_data'] = 118
  params['max_depth'] = 23
  params['boosting_type'] = 'goss'
  params['min_child_weight'] = 2.0584825310849335
  params['max_drop'] = 97
  params['use_best_model'] = True
  params['metric'] = 'rmse'
  lgbm_model = lgb.LGBMRegressor(**params)

  xgb_params = {}
  xgb_params['objective'] = 'reg:squarederror'
  xgb_params['n_estimators'] = 30000
  xgb_params['max_depth'] = 16
  xgb_params['min_child_weight'] = 50
  xgb_params['gamma'] = 7.5
  xgb_params['learning_rate'] = 0.001
  xgb_params['colsample_bytree'] = 0.75
  xgb_params['lambda'] = 0.49823098658698656
  xgb_params['alpha'] = 0.04170536352735257
  xgb_params['subsample'] = 0.6
  xgb_params['random_state'] = 42
  xgb_params['use_best_model'] = True
  xgb_params['tree_method'] = 'gpu_hist'
  xgb_params['gpu_id'] = 0
  xgb_model = XGBRegressor(**xgb_params)

  cat_params = {} #initialize parameters
  cat_params['random_state'] = 42
  cat_params['iterations'] = 30000
  cat_params['subsample'] = 0.9699172985498098
  cat_params['max_depth'] = 7
  cat_params['l2_leaf_reg'] = 0.4829028252738887
  cat_params['objective'] = 'RMSE'
  cat_params['bootstrap_type'] = 'Bernoulli'
  cat_params['min_data_in_leaf'] = 70
  cat_params['learning_rate'] = 0.001
  cat_params['learning_rate'] = 0.01
  cat_params['task_type'] = 'GPU'
  cat_params['use_best_model'] = True
  cat_model = catboost.CatBoostRegressor(**cat_params)

  lgbm_ratio = 5
  xgb_ratio = 1
  cat_ratio = 4
  # score = cv_evaluation_regression(X, model)
  score, lgbm_pred, xgb_pred, cat_pred, lgb_train, xgb_train, cat_train  = cv_evaluation_regression2(X, lgbm_model, xgb_model, cat_model, test_data)

  return score, lgbm_pred, xgb_pred, cat_pred, lgb_train, xgb_train, cat_train 

In [29]:
score, lgbm_pred, xgb_pred, cat_pred, lgb_train, xgb_train, cat_train  = objective(train_df, testset2)

LGBM
[LightGBM] [Warning] Unknown parameter: use_best_model
[LightGBM] [Warning] min_data_in_leaf is set with min_data=118, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=118
[LightGBM] [Warning] feature_fraction is set=0.8073718294893066, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8073718294893066
Training until validation scores don't improve for 500 rounds
[2000]	valid_0's rmse: 0.851801
[4000]	valid_0's rmse: 0.845709
[6000]	valid_0's rmse: 0.84391
[8000]	valid_0's rmse: 0.84332
Early stopping, best iteration is:
[9153]	valid_0's rmse: 0.843186
XGB
[0]	validation_0-rmse:7.00636
Will train until validation_0-rmse hasn't improved in 250 rounds.
[500]	validation_0-rmse:4.30464
[1000]	validation_0-rmse:2.69932
[1500]	validation_0-rmse:1.77479
[2000]	validation_0-rmse:1.27342
[2500]	validation_0-rmse:1.02635
[3000]	validation_0-rmse:0.917271
[3500]	validation_0-rmse:0.872775
[4000]	validation_0-rmse:0.855292
[4500]	validation_0-rmse:0.

In [30]:
preds = pd.concat([pd.DataFrame(lgbm_pred), pd.DataFrame(xgb_pred), pd.DataFrame(cat_pred)], axis = 1)
x_trains = pd.concat([pd.DataFrame(lgb_train), pd.DataFrame(xgb_train), pd.DataFrame(cat_train)], axis = 1)

In [31]:
preds.columns = ['LGBM', "XGB", 'CAT']
x_trains.columns = ['LGBM', "XGB", 'CAT']

In [32]:
preds.to_csv('predictions.csv')
!cp predictions.csv "drive/My Drive/"

x_trains.to_csv('trains.csv')
!cp trains.csv "drive/My Drive/"